<a href="https://colab.research.google.com/github/daryllman/walmart-scraper/blob/master/scraper_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load in the necessary libraries

In [ ]:
import requests
from bs4 import BeautifulSoup as bs #pip install beautifulsoup4

## Load sample webpage content



In [ ]:
# Load the sample webpage
#base_url = 'https://www.walgreens.com/store/store/category/productlist.jsp?N=359451&Eon=359451&No=98'
base_url = 'https://www.walgreens.com/store/store/category/productlist.jsp?No=24&N=359451&Eon=359451'

r = requests.get(base_url)

# Convert to a beautiful soup object
soup = bs(r.content)

In [ ]:
#print(r.content)
#print(soup)
print(soup.prettify())

## Extract Item URLs from page


In [ ]:
# Extract Item divs from page
item_divs = soup.findAll('div', id=lambda x: x and x.startswith('productcardprod'))
#item_divs = soup.findAll('div', id=lambda x: x and x.startswith('product-row-'))
print(item_divs)
print(len(item_divs))

[<div class="col-xl-3 col-lg-3 col-md-6 col-sm-6 col-6 product-column pr0" data-reactid="968" id="productcardprod6296304"><div class="wag-product-cards wag-pos-rel" data-reactid="969"><div class="wag-prod-card-btm wag-pos-rel" data-reactid="970" id="productcardheightprod6296304"><div class="beagleprod" data-reactid="971"></div><div class="wag-product-card-details wag-product-card-width_b" data-reactid="972" id="productcard1"><div class="text-center mb20" data-reactid="973"><a data-reactid="974" href="/store/c/lorna-doone-shortbread-cookies-shortbread/ID=prod6296304-product" id="productOmniSelectcompare_sku6250495" title="Lorna Doone Shortbread Cookies Shortbread (1.5 oz )"><img alt="Lorna Doone Shortbread Cookies Shortbread" class="img-responsive wag-familyoffer-productsimg imglazy " data-reactid="975" id="secImgcompare_sku6250495" src="//pics.drugstore.com/prodimg/560885/155.jpg"/><span class="sr-only" data-reactid="976"><!-- react-text: 977 -->Lorna Doone Shortbread Cookies Shortbrea

In [ ]:
import re
item_divs = soup.find_all('div', id=re.compile("productcardprod"))
#print(item_divs)
url_links = [item.find('a', id=re.compile("productOmniSelectcompare")).get('href', '') for item in item_divs]
print(url_links)

['/store/c/lorna-doone-shortbread-cookies-shortbread/ID=prod6296304-product', '/store/c/golden-grahams-cereal/ID=prod6383175-product', '/store/c/maxwell-house-ground-coffee-master-blend/ID=prod6118209-product', '/store/c/cheerios-honey-nut-medley-crunch-cereal/ID=prod6213455-product', "/store/c/nescafe-taster's-choice-instant-coffee-decaf-house-blend/ID=prod6334522-product", '/store/c/raisin-bran-cereal/ID=prod6112661-product', '/store/c/cheez-it-baked-snack-crackers/ID=prod6111671-product']


## Extract Item details from one item url

In [ ]:
# Extract item detail from one product
base_url = "https://www.walgreens.com"
item1 = url_links[0]
item1_url = "{}{}".format(base_url,item1)
print(item1_url)



https://www.walgreens.com/store/c/lorna-doone-shortbread-cookies-shortbread/ID=prod6296304-product


In [ ]:
r = requests.get(item1_url)

# Convert to a beautiful soup object
item_soup = bs(r.content)
item_dict = dict()
item_dict["categoricalFeatures"] = {}
item_dict["textFeatures"] = {}
item_dict["numericalFeatures"] = {}


In [ ]:
# Get Name
item_name = item_soup.find(id='productName').find('span').getText()
item_dict["textFeatures"]["name"] = item_name

In [ ]:
# Get Image URL
item_img_url = item_soup.find(id='productImg').get('src')
item_dict["image_url"] = item_img_url

In [ ]:
# Get Brand
item_brand = item_soup.find('a',attrs={'class':'brand-title'}).find('strong').getText()
item_dict["categoricalFeatures"]["brand"] = {"values":[item_brand]}

In [ ]:
# Get Categories
item_cat_list = item_soup.find('ul',attrs={'class':'nav__bread-crumbs'}).findAll('li')
item_cat_list_formatted = [cat.getText() for cat in item_cat_list]
remaining_list = item_cat_list_formatted[item_cat_list_formatted.index("Grocery"):]
item_dict["categoricalFeatures"]["category"] = {"values":remaining_list}

In [ ]:
# Get Price
price_list = item_soup.find('div',attrs={'id':'product'}).find('span',attrs={'class':'product__price'}).findAll()
price_list_formatted = [price.getText() for price in price_list]
price_string = "{}{}".format(price_list_formatted[0], '.'.join(price_list_formatted[1:]))
item_dict["numericalFeatures"]["price"] = price_string

In [ ]:
# Get Rating
rating_val = item_soup.find('div', id="prodBV").find('span').get('aria-label').split()[0]
item_dict["numericalFeatures"]["rating"] = rating_val

In [ ]:
# Get Description
item_desc_list = item_soup.find('li', id="prodDesc").find('div',attrs={'class':'view-more-content__height'}).find_all(['li','p'])
item_desc_list_formatted = [desc.getText() for desc in item_desc_list]
item_desc_string = ', '.join(item_desc_list_formatted)
item_dict["textFeatures"]["description"] = item_desc_string

In [ ]:
# Get Ingredients
item_ingrd = item_soup.find('li', id="Ingredients").find('span').getText()
item_dict["textFeatures"]["ingredients"] = item_ingrd

In [ ]:
# Get Nutrional Information
def format_nutri_servings_info(bs_div_item):
  full_str = bs_div_item.getText()
  key = bs_div_item.strong.getText().strip()
  val = full_str.lstrip(key).strip()
  return "{}: {}".format(key,val)

def format_nutri_body_info(bs_tr_item):
  intended_content = bs_tr_item.find('td')
  full_str = intended_content.getText()
  key = intended_content.strong.getText().strip()
  val = full_str.lstrip(key).strip()
  return "{}: {}".format(key,val)

item_nutri_list = []
item_nutri = item_soup.find('div',attrs={'class':'table-container nutrition'})
item_nutri_servings = item_nutri.find_all('div')
item_nutri_servings_formatted = [format_nutri_servings_info(item_s) for item_s in item_nutri_servings]
print(item_nutri_servings_formatted)

item_nutri_body = item_nutri.find('tbody').find_all('tr')
item_nutri_body_formatted = [format_nutri_body_info(item_s) for item_s in item_nutri_body]
print(item_nutri_body_formatted)

item_nutri_list = item_nutri_list + item_nutri_servings_formatted + item_nutri_body_formatted
print(item_nutri_list)

item_dict["textFeatures"]["nutritionInformation"] = {"values":item_nutri_list}

In [ ]:
# Get Product ID
# UPC no. if available, if not product id
item_upc = item_soup.find('p',attrs={'class':'universal-Item-code'}).getText()
item_upc="".join(item_upc.replace(":","").split())
print(item_upc)
item_code = item_soup.find('p',attrs={'class':'universal-product-code'}).getText()
item_code = item_code.split()[-1]
print(item_code)

UPC04400004250
918167


In [ ]:
import re
prod_id = item1_url.split('/')[-1]
prod_id_formatted = re.split('=|-',prod_id )[1]
prod_id_formatted

'prod6296304'

Further formatting of Description text

In [ ]:
samplestr ="ted\u00a0& Approve.*\u00a0 Patented Fermentation Process.\u00a0 \"All-Natural\".\u00a0 Non-GMO., This"
#samplestr_formatted = samplestr.replace("\u00a0A", " ")
#samplestr_formatted = samplestr_formatted.replace("\\", " ")
samplestr_formatted = "".join(re.split('\u00a0A|..',samplestr ))
print(samplestr_formatted)

s
